In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from torch.func import vmap, jacrev
from tqdm import tqdm
import os
import random
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import Callback
import math
from pydmd import DMD
from sklearn.preprocessing import MinMaxScaler
import warnings
from torchdiffeq import odeint

In [2]:
class ODEFunc(nn.Module):
    def __init__(self, dim, hidden_dim, n_layers, input_dim=0, dropout=0.0):
        super().__init__()
        self.input_dim = input_dim
        self.use_control = input_dim > 0

        in_dim = dim + input_dim if self.use_control else dim
        layers = [nn.Linear(in_dim, hidden_dim), nn.Tanh()]
        for _ in range(n_layers):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.Tanh())
            layers.append(nn.Dropout(dropout))
        layers.append(nn.Linear(hidden_dim, dim))
        self.net = nn.Sequential(*layers)

        self._initialize_weights()

    def forward(self, t, x, u=None):
        if self.use_control:
            if u is None:
                raise ValueError("Control input u must be provided.")
            x_input = torch.cat([x, u], dim=-1)
        else:
            x_input = x
        return self.net(x_input)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

class NeuralODE(nn.Module):
    def __init__(self, odefunc, t=torch.tensor([0.0, 1.0]), solver='rk4'):
        super().__init__()
        self.odefunc = odefunc
        self.integration_time = t
        self.solver = solver

    def forward(self, x, u=None):
        t = self.integration_time.to(x.device)

        if self.odefunc.use_control:
            if u is None:
                raise ValueError("This NeuralODE expects control input u, but got None.")
            out = odeint(lambda t, x_: self.odefunc(t, x_, u), x, t, method=self.solver)
        else:
            out = odeint(lambda t, x_: self.odefunc(t, x_), x, t, method=self.solver)

        return out[-1], 0.0

In [3]:
def node_rollout(model, X, U=None):
    X_pred_list = []
    X_true_list = []

    B, T, dim = X.shape

    for i in range(B):
        Xi = X[i]         # [T, dim]
        Xi0 = Xi[:-1]     # [T-1, dim]
        Xi1 = Xi[1:]      # [T-1, dim]

        if U is not None:
            Ui = U[i]
            Ui0 = Ui[:-1]

        Xi_pred = []
        for t in range(T - 1):
            xt = Xi0[t:t+1]      # [1, dim]
            if U is not None:
                ut = Ui0[t:t+1]  # [1, input_dim]
                xt_next, _ = model(xt, ut)
            else:
                xt_next, _ = model(xt)
            Xi_pred.append(xt_next)

        Xi_pred = torch.cat(Xi_pred, dim=0).T     # [dim, T-1]
        Xi_true = Xi1.T                           # [dim, T-1]

        X_pred_list.append(Xi_pred)
        X_true_list.append(Xi_true)

    X_pred_all = torch.cat(X_pred_list, dim=-1)   # [dim, total]
    X_true_all = torch.cat(X_true_list, dim=-1)
    return X_true_all, X_pred_all

In [4]:
class TrainModel(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3, path="model_checkpoint_NODE_control"):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = nn.MSELoss()
        self.best_val_loss = float('inf')
        self.validation_outputs = []
        self.train_losses = []
        self.path = path + '.ckpt'

    def forward(self, x, u):
        return self.model(x, u)

    def training_step(self, batch, batch_idx):
        X_batch, U_batch = batch  # [B, T, dim], [B, T, input_dim]
        X_true, X_pred = node_rollout(self.model, X_batch, U_batch)

        loss = self.criterion(X_true, X_pred)
        self.log('train_loss', loss, on_step=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        X_batch, U_batch = batch
        X_true, X_pred = node_rollout(self.model, X_batch, U_batch)

        valid_loss = self.criterion(X_pred, X_true)
        self.validation_outputs.append(valid_loss)
        self.log('val_loss', valid_loss)
        return valid_loss

    def test_step(self, batch, batch_idx):
        X_batch, U_batch = batch
        X_true, X_pred = node_rollout(self.model, X_batch, U_batch)

        test_loss = self.criterion(X_pred, X_true)
        self.log('test_loss', test_loss)
        return test_loss
    
    def on_fit_start(self):
        if self.trainer.is_global_zero:
            if os.path.exists("loss_log.txt"):
                os.remove("loss_log.txt")
            if os.path.exists(self.path):
                os.remove(self.path)

    def on_validation_epoch_end(self):
        avg_val_loss = torch.stack(self.validation_outputs).mean()
        self.log('avg_val_loss', avg_val_loss)
        self.validation_outputs.clear()
        print(f"Validation loss: {avg_val_loss}")
        with open("loss_log.txt", "a") as f:
            f.write(f"{avg_val_loss.item()}\n")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.92)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
            },
            "gradient_clip_val": 1.0,
            "gradient_clip_algorithm": "norm",
        }

In [5]:
dim = 2  
hidden_dim = 20  
input_dim = 1
n_blocks = 5  
n_layers = 10
n_feature = 18
rank = 50
batch_size = 64
batch_size = 512
n_train = 10000
n_valid = 1000
n_test = 1000
dropout = 0
num_epochs = 100  
lamb = 0
learning_rate = 1e-3 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
X_train = pd.read_csv('limit_X_train.csv', header=None).values
X_valid = pd.read_csv('limit_X_valid.csv', header=None).values
X_test = pd.read_csv('limit_X_test.csv', header=None).values
U_train = pd.read_csv('limit_U_train.csv', header=None).values
U_valid = pd.read_csv('limit_U_valid.csv', header=None).values
U_test = pd.read_csv('limit_U_test.csv', header=None).values

length = X_train.shape[1] // n_train
HX_train = []
HU_train = []
for i in range(n_train):
    HX_train.append(X_train[:, i*length:(i+1)*length])
    HU_train.append(U_train[:, i*length:(i+1)*length])
HX_train = np.stack([HX_train[idx].T for idx in range(n_train)], axis=0)
HU_train = np.stack([HU_train[idx].T for idx in range(n_train)], axis=0)
HX_valid = []
HU_valid = []
for i in range(n_valid):
    HX_valid.append(X_valid[:, i*length:(i+1)*length])
    HU_valid.append(U_valid[:, i*length:(i+1)*length])
HX_valid = np.stack([HX_valid[idx].T for idx in range(n_valid)], axis=0)
HU_valid = np.stack([HU_valid[idx].T for idx in range(n_valid)], axis=0)
train_dataset = TensorDataset(torch.tensor(HX_train, dtype=torch.float32), torch.tensor(HU_train, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_dataset = TensorDataset(torch.tensor(HX_valid, dtype=torch.float32), torch.tensor(HU_valid, dtype=torch.float32))
valid_loader = DataLoader(valid_dataset, batch_size=99999, shuffle=True, num_workers=0, pin_memory=True)

In [ ]:
model_path = "model_checkpoint_limit_node"
odefunc = ODEFunc(dim=dim, hidden_dim=hidden_dim, n_layers=n_layers)
neural_ode = NeuralODE(odefunc=odefunc, t=torch.tensor([0.0, 1.0]), solver="euler")
lightning_model = TrainModel(model=neural_ode, learning_rate=learning_rate, path=model_path)
checkpoint_callback = ModelCheckpoint(
    monitor="avg_val_loss",
    dirpath="./",
    filename=model_path,
    save_top_k=1,
    mode="min",
)
trainer = pl.Trainer(accelerator="gpu", devices=4, max_epochs=num_epochs, callbacks=[checkpoint_callback])
trainer.fit(lightning_model, train_loader, train_loader)

In [7]:
odefunc = ODEFunc(dim=dim, hidden_dim=hidden_dim, n_layers=n_layers)
neural_ode = NeuralODE(odefunc=odefunc, t=torch.tensor([0.0, 1.0]), solver="euler")
path = "model_checkpoint_limit_node.ckpt"
lightning_model = TrainModel(model=neural_ode, learning_rate=learning_rate, path=path)
trainer = pl.Trainer(accelerator="gpu", devices=4, strategy="ddp_notebook", max_epochs=num_epochs)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [8]:
length = X_test.shape[1] // n_test
HX_test = []
HU_test = []
for i in range(n_test):
    HX_test.append(X_test[:, i*length:(i+1)*length])
    HU_test.append(U_test[:, i*length:(i+1)*length])
HX_test = np.stack([HX_test[idx].T for idx in range(n_test)], axis=0)
HU_test = np.stack([HU_test[idx].T for idx in range(n_test)], axis=0)
test_dataset = TensorDataset(torch.tensor(HX_test, dtype=torch.float32), torch.tensor(HU_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=9999, shuffle=True)

In [9]:
warnings.filterwarnings("ignore")
trainer.test(lightning_model, dataloaders=test_loader)

You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/4
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. 

Testing DataLoader 0: 100%|██████████| 1/1 [00:10<00:00,  0.10it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.05708123371005058
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.05708123371005058}]